In [1]:
import shutil
import subprocess
from datetime import timedelta
from datetime import datetime
from pathlib import Path
import json
import math

In [2]:
cmd_templ = "ffprobe -v error -show_entries format=duration -of default=noprint_wrappers=1:nokey=1"

In [19]:
video_dirs = Path("d:/LangOn/reform/legislative_old/").iterdir()
asr_paths = list(Path("d:/LangOn/reform/asr_old").glob("*.json"))
new_asr_dir = Path("d:/LangOn/reform/asr")
failed = []
for committee_dir in video_dirs:
    metadata_path = list(committee_dir.glob('**/*.json'))[0]
    with metadata_path.open("r", encoding="UTF-8") as fin:
        metadata = json.load(fin)
    for video_path in committee_dir.glob("**/*.mp4"):
        video_length = subprocess.check_output([*cmd_templ.split(), video_path]).decode()
        video_length = float(video_length.strip())        
        video_id = video_path.stem.split("-")[-1]
        video_idx = [idx for (idx, x) in enumerate(metadata)
                     if x["video_id"] == video_id][0] - 1
        if video_idx < 0:
            print(video_path, "is not available")
            failed.append(video_path.parent / meta_name)
            continue
        meta_entry = metadata[video_idx]
        length_in_meta = datetime.strptime(meta_entry["clip_length"], "%H:%M:%S")
        length_in_meta = (length_in_meta - datetime(1900,1,1)).seconds
        meta_name = "{}-{}".format(
                    meta_entry["speaker"], meta_entry["video_id"])
        if abs(video_length-length_in_meta)<1.:
            # print(video_path.name, meta_name, video_length, length_in_meta)        
            asr_path = [x for x in asr_paths if video_path.stem in str(x)]
            if asr_path:
                shutil.copy(asr_path[0], new_asr_dir/(meta_name+".gasr.json"))
        else:
            failed.append(video_path.parent / meta_name)
            # print(video_path, "alignment failed ({},{})".format(video_length, length_in_meta))    

d:\LangOn\reform\legislative_old\交通\2021-03-15\邱志偉-127921.mp4 is not available
d:\LangOn\reform\legislative_old\內政\2021-03-15\陳椒華-127940.mp4 is not available
d:\LangOn\reform\legislative_old\司法及法制\2021-03-15\賴香伶-127920.mp4 is not available
d:\LangOn\reform\legislative_old\外交及國防\2021-03-15\呂玉玲-127919.mp4 is not available
d:\LangOn\reform\legislative_old\教育及文化\2021-03-15\楊瓊瓔-127929.mp4 is not available
d:\LangOn\reform\legislative_old\社會福利及衛生環境\2021-03-17\陳椒華-128106.mp4 is not available
d:\LangOn\reform\legislative_old\經濟\2021-03-15\賴香伶-127944.mp4 is not available
d:\LangOn\reform\legislative_old\財政\2021-03-15\江永昌-127912.mp4 is not available


In [20]:
Path("../../data/aligned_failed.txt").write_text("\n".join(str(x) for x in failed), encoding="UTF-8")

1382